In [1]:
import os
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

from mpl_toolkits.basemap import Basemap

import numpy as np
import pandas as pd

# Loading

In [2]:
TRAIN_PATH = "data/download/train.csv"
TEST_PATH = "data/download/test.csv"

PROC_DIR = "data/processed_data"

In [3]:
# number of rows of train
!wc -l {TRAIN_PATH}

 55423856 data/download/train.csv


In [4]:
# number of rows of test
!wc -l {TEST_PATH}

    9914 data/download/test.csv


In [5]:
# let's read the first few rows
train_df = pd.read_csv(TRAIN_PATH, nrows=5)
train_df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
key                  5 non-null object
fare_amount          5 non-null float64
pickup_datetime      5 non-null object
pickup_longitude     5 non-null float64
pickup_latitude      5 non-null float64
dropoff_longitude    5 non-null float64
dropoff_latitude     5 non-null float64
passenger_count      5 non-null int64
dtypes: float64(5), int64(1), object(2)
memory usage: 400.0+ bytes


In [7]:
pd.to_datetime(train_df["key"])

0   2009-06-15 17:26:21.000000100
1   2010-01-05 16:52:16.000000200
2   2011-08-18 00:35:00.000000490
3   2012-04-21 04:30:42.000000100
4   2010-03-09 07:51:00.000000135
Name: key, dtype: datetime64[ns]

In [8]:
pd.to_datetime(train_df["pickup_datetime"])

0   2009-06-15 17:26:21+00:00
1   2010-01-05 16:52:16+00:00
2   2011-08-18 00:35:00+00:00
3   2012-04-21 04:30:42+00:00
4   2010-03-09 07:51:00+00:00
Name: pickup_datetime, dtype: datetime64[ns, UTC]

Columns `key` and and `pickup_datetime` seem to be the same timestamp, except that `pickup_datetime` has second precision whereas `key` has nano second precision. `key` is used to uniquely identify each row and hence not important for data analysis and modeling. We will drop this column in the train set. `key` is still needed for the test set to write out the Kaggle submission file.

`train.csv` has a large number of rows (more than 55 million) so it may take a long time to read it into a dataframe. To quickly read the data and reduce its size, we follow this post: https://www.kaggle.com/szelee/how-to-import-a-csv-file-of-55-million-rows .

In [9]:
def load_nyc_taxi_fare(path, col_types, chunksize=None, 
                       datetime_format="%Y-%m-%d %H:%M:%S UTC",
                       convert_to_timezone=None):
    chunk_iter = pd.read_csv(path, usecols=col_types.keys(), dtype=col_types, chunksize=chunksize)
    
    if chunksize is None:
        chunk_iter["pickup_datetime"] = pd.to_datetime(chunk_iter["pickup_datetime"], 
                                                       utc=True, format=datetime_format)
        # convert to different timezone
        if convert_to_timezone is not None:
            chunk_iter["pickup_datetime"] = chunk_iter["pickup_datetime"].dt.tz_convert(convert_to_timezone)
        return chunk_iter
    
    df_list = []
    # use tqdm to monitor progress
    # It would take extremely long time if format were not used.
    for df_chunk in tqdm(chunk_iter):
        df_chunk["pickup_datetime"] = pd.to_datetime(df_chunk["pickup_datetime"], 
                                                     utc=True, format=datetime_format)
        # convert to different timezone
        if convert_to_timezone is not None:
            df_chunk["pickup_datetime"] = df_chunk["pickup_datetime"].dt.tz_convert(convert_to_timezone)
        
        df_list.append(df_chunk)
        
    return pd.concat(df_list)

In [ ]:
train_types = {"fare_amount": "float32",
              "pickup_datetime": "str", 
              "pickup_longitude": "float32",
              "pickup_latitude": "float32",
              "dropoff_longitude": "float32",
              "dropoff_latitude": "float32",
              "passenger_count": "uint8"}
train_df = load_nyc_taxi_fare(TRAIN_PATH, train_types, chunksize=5_000_000, convert_to_timezone='US/Eastern')
train_df.head()

0it [00:00, ?it/s]

In [ ]:
train_df.shape

In [ ]:
train_df.info(memory_usage="deep")

In [ ]:
test_types = train_types.copy()
test_types.pop("fare_amount")
test_types["key"] = "str"

test_df = load_nyc_taxi_fare(TEST_PATH, test_types, convert_to_timezone='US/Eastern')
test_df.head()

# Cleaning and EDA

In [ ]:
train_df.describe()

Is there any Null?

In [ ]:
train_df.isna().sum()

Since the number of rows containing null is very small compared to the total number of rows, we decide to drop them.

In [ ]:
print("shape before dropping:", train_df.shape)
train_df = train_df.dropna(axis=0, how="any")
print("shape after dropping:", train_df.shape)

## Cleaning up `fare_amount`

There are negative fare amounts. Let's count how there are in the training set.

In [ ]:
print("Number of negative values of fare_amount: %d " % (train_df["fare_amount"] < 0).sum())

Since negative values of `fare_amount` do not make sense, we will drop them.

In [ ]:
print("Shape before dropping:", train_df.shape)
train_df = train_df[train_df["fare_amount"] >= 0]
print("Shape after dropping:", train_df.shape)

Very large values of `fare_amount` may also be problematic. Let's look the histogram.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
train_df["fare_amount"].plot(kind="hist", ax=ax[0], bins=100)
ax[0].set_xlabel("fare_amount (USD)")
ax[0].set_title("all data")

train_df.loc[train_df["fare_amount"] < 200, "fare_amount"].plot(kind="hist", bins=100, ax=ax[1])
ax[1].set_xlabel("fare_amount (USD)")
ax[1].set_title("data with fare_amount < 200")

In [ ]:
for fare in [100, 200, 500, 1000, 5000, 10000]:
    print("Number of rows with fare_amount greater than $%d is %d" %(fare, (train_df["fare_amount"] > fare).sum()))

There are few values greater than $200. Latter we will see if these expensive trips correspond to long distances and decide whether we will drop them.

## Clean up pickup and dropoff location data

In [ ]:
train_df.describe()

Latitude ranges from -90$^o$ to 90$^o$ and longitude from -180$^o$ to 180$^o$. Values outside these ranges do not make sense and we will drop them.

In [ ]:
def drop_wrong_gps_coor(df):
    print("Shape before dropping:", df.shape)
    df = df[(df["pickup_longitude"] >= -180) & (df["pickup_longitude"] <= 180)]
    df = df[(df["dropoff_longitude"] >= -180) & (df["dropoff_longitude"] <= 180)]
    
    df = df[(df["pickup_latitude"] >= -90) & (df["pickup_latitude"] <= 90)]
    df = df[(df["dropoff_latitude"] >= -90) & (df["dropoff_latitude"] <= 90)]
    print("Shape after dropping:", df.shape)
    return df

In [ ]:
train_df = drop_wrong_gps_coor(train_df)

In [ ]:
train_df.describe()

By googling we know that the geo-coordinate of the center of New York City is (40.7128$^o$ N, 74.0060$^o$ W) which means `latitute` = 40.7128$^o$ and `longitude` = -74.0060$^o$. We expect the pickup and dropoff coordinates in the train data set vary by one or two degrees at most from the center. However, the min and max values of the `longitude` and `latitude` are very extreme. 

We will focus on the region around New York City. We follow this post: https://www.kaggle.com/breemen/nyc-taxi-fare-data-exploration and define a bounding box from GPS coordinates in the test set.

In [ ]:
LON_MIN = np.min([test_df["pickup_longitude"].min(), test_df["dropoff_longitude"].min()])
LON_MAX = np.max([test_df["pickup_longitude"].max(), test_df["dropoff_longitude"].max()])

LAT_MIN = np.min([test_df["pickup_latitude"].min(), test_df["dropoff_latitude"].min()])
LAT_MAX = np.max([test_df["pickup_latitude"].max(), test_df["dropoff_latitude"].max()])

print("LON_MIN = %0.5f" % LON_MIN)
print("LON_MAX = %0.5f" % LON_MAX)
print("LAT_MIN = %0.5f" % LAT_MIN)
print("LAT_MAX = %0.5f" % LAT_MAX)

Let's see how many data points are outside the bounding box.

In [ ]:
def is_pickup_inside(df, lon_min, lon_max, lat_min, lat_max):
    lat_inside = (df["pickup_latitude"] >= lat_min) & (df["pickup_latitude"] <= lat_max)
    lon_inside = (df["pickup_longitude"] >= lon_min) & (df["pickup_longitude"] <= lon_max)
    return lat_inside & lon_inside

def is_dropoff_inside(df, lon_min, lon_max, lat_min, lat_max):
    lat_inside = (df["dropoff_latitude"] >= lat_min) & (df["dropoff_latitude"] <= lat_max)
    lon_inside = (df["dropoff_longitude"] >= lon_min) & (df["dropoff_longitude"] <= lon_max)
    return lat_inside & lon_inside

def is_inside(df, lon_min, lon_max, lat_min, lat_max):
    """both pickup and dropoff are inside"""
    p_in = is_pickup_inside(df, lon_min, lon_max, lat_min, lat_max)
    d_in = is_dropoff_inside(df, lon_min, lon_max, lat_min, lat_max)
    return p_in & d_in

In [ ]:
inside = is_inside(train_df, LON_MIN, LON_MAX, LAT_MIN, LAT_MAX)
outside = ~inside
print("%0.3f%% of points outside the bounding box" % (outside.mean() * 100))

Let's increase the bounding box a little bit to include more points in the box.

In [ ]:
for deg in [0.05, 0.1, 0.2, 0.5, 1, 2, 5]:
    lat_min = LAT_MIN - deg / 2
    lat_max = LAT_MAX + deg / 2
    lon_min = LON_MIN - deg / 2
    lon_max = LON_MAX + deg / 2
    
    inside = is_inside(train_df, lon_min, lon_max, lat_min, lat_max)
    outside = ~inside
    print("When increasing each side by %0.3f deg, %0.3f%% of points are outside." % (deg, outside.mean()*100))

When increasing the boudning box, not a lot of points actually move inside. This means that most of the points outside the box are really far away. There are about 2% of the points having both pickup and dropoff locations are far from New York City. Let's look at them on the map.

In [ ]:
def create_nyc_map(ax, lon_min, lon_max, lat_min, lat_max, alpha=1., resolution="h"):
    # projection="cyl" or projection="lcc"
    bmap = Basemap(projection="cyl", resolution=resolution, ax=ax,
                   llcrnrlat=lat_min, urcrnrlat=lat_max, 
                   llcrnrlon=lon_min, urcrnrlon=lon_max,
                   lat_0=(lat_min + lat_max) / 2, lon_0=(lon_min + lon_max) / 2)
    
    bmap.drawmapboundary(fill_color='aqua', ax=ax, zorder=0)
    bmap.fillcontinents(color="coral", lake_color='aqua', alpha=alpha, ax=ax, zorder=1)
    bmap.drawstates(color='gray', ax=ax, zorder=2)
    
    return bmap

def plot_coor_on_map(bmap, lons, lats, marker="o", s=1, c="k", alpha=1, label=None):
    bmap.scatter(lons, lats, latlon=True, marker=marker, s=s, c=c, zorder=3, alpha=alpha, label=label)
    return bmap

In [ ]:
df_tmp = train_df.sample(frac=0.02, random_state=210)
print("df_tmp shape", df_tmp.shape)
inside_idx = is_pickup_inside(df_tmp, LON_MIN, LON_MAX, LAT_MIN, LAT_MAX)

sns.set(font_scale=1)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,6))

bmap = create_nyc_map(ax[0], LON_MIN, LON_MAX, LAT_MIN, LAT_MAX)
bmap = plot_coor_on_map(bmap,  
                        df_tmp["pickup_longitude"].values,
                        df_tmp["pickup_latitude"].values,
                        marker="o", s=1, c="k", label=None)
ax[0].set_title("Bounding box determined from test set")

bmap = create_nyc_map(ax[1], LON_MIN-2.5, LON_MAX+2.5, LAT_MIN-2.5, LAT_MAX+2.5)
bmap = plot_coor_on_map(bmap,  
                        df_tmp.loc[inside_idx, "pickup_longitude"].values,
                        df_tmp.loc[inside_idx, "pickup_latitude"].values,
                        marker="o", s=1, c="k", label="inside")
bmap = plot_coor_on_map(bmap,  
                        df_tmp.loc[~inside_idx, "pickup_longitude"].values,
                        df_tmp.loc[~inside_idx, "pickup_latitude"].values,
                        marker="o", s=1, c="g", label="outside")
ax[1].set_title("Bounding box enlarged by 5 deg each side")
ax[1].legend()

del df_tmp

The area of bounding box on the right is really large. Since we are interested in taxi rides near New York City, we will use the bounding box given by the test set enlarged by 1 degree each side.

In [ ]:
inside_idx = is_inside(train_df, LON_MIN-0.5, LON_MAX+0.5, LAT_MIN-0.5, LAT_MAX+0.5)
print("Shape before dropping:", train_df.shape)
train_df = train_df[inside_idx]
print("Shape after dropping:", train_df.shape)

In [ ]:
df_tmp = train_df.sample(frac=0.1, random_state=210)
sns.set(font_scale=1)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,6))

bmap = create_nyc_map(ax, LON_MIN-0.5, LON_MAX+0.5, LAT_MIN-0.5, LAT_MAX+0.5)
bmap = plot_coor_on_map(bmap,  
                        df_tmp["pickup_longitude"].values,
                        df_tmp["pickup_latitude"].values,
                        marker="o", s=1, c="k", label=None)
ax.set_title("Bounding box enlarged by 1 deg each side")

del df_tmp

There are a lot of points on water. This makes little sense. Let's see if this also happens for the test set.

In [ ]:
sns.set(font_scale=1)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6,6))

bmap = create_nyc_map(ax, LON_MIN, LON_MAX, LAT_MIN, LAT_MAX)
bmap = plot_coor_on_map(bmap,  
                        test_df["pickup_longitude"].values,
                        test_df["pickup_latitude"].values,
                        marker="o", s=2, c="k", label=None)
ax.set_title("Pickup coordinates of test set")

It happens the test set but rare. Points on water are much closer to shore which may be acceptable. Before deciding whether to remove all the taxi rides which start (pickup) and/or end (dropoff) on water from the training set, let's see if the target variable `fare_amount` is related to whether the pickup or dropoff coordinate in on water. 

In [ ]:
def is_on_land(lons, lats):
    
    lon_min = np.min(lons) - 0.01
    lon_max = np.max(lons) + 0.01
    
    lat_min = np.min(lats) - 0.01
    lat_max = np.max(lats) + 0.01
    
    lon_0 = (lon_min + lon_max) / 2
    lat_0 = (lat_min + lat_max) / 2
    
    # projection="cyl" or projection="lcc"
    bm = Basemap(projection="cyl", resolution="h", 
                 llcrnrlat=lat_min, urcrnrlat=lat_max, 
                 llcrnrlon=lon_min, urcrnrlon=lon_max,
                 lon_0=lon_0, lat_0=lat_0)
    
    xpt, ypt = bm(lons, lats)
    vec_is_land = np.vectorize(bm.is_land)
    return vec_is_land(xpt, ypt)

def is_pickup_on_land(df):
    return is_on_land(df["pickup_longitude"].values, df["pickup_latitude"].values)

def is_dropoff_on_land(df):
    return is_on_land(df["dropoff_longitude"].values, df["dropoff_latitude"].values)

def is_both_pickup_or_dropoff_on_land(df):
    return is_pickup_on_land(df) & is_dropoff_on_land(df)

Determining whether pickup and dropoff locations are on land or water takes a long time. So will do that for only 2% of the training data.

In [ ]:
create_csv = False

if create_csv:
    train_df_small = train_df.sample(frac=0.02, random_state=210)
    train_df_small["on_land"] = is_both_pickup_or_dropoff_on_land(train_df_small)
    train_df_small.to_csv(os.path.join(PROC_DIR, "train_df_small.csv"))
    print(train_df_small.shape)
    
else:
    train_df_small = pd.read_csv(os.path.join(PROC_DIR, "train_df_small.csv"))
    print(train_df_small.shape)

In [ ]:
print("There are %0.3f%% of points on water." % ((1 - train_df_small["on_land"].mean())*100) )

In [ ]:
on_land = train_df_small["on_land"]
on_water = ~on_land

sns.set(font_scale=1)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))

bmap = create_nyc_map(ax[0], LON_MIN-0.5, LON_MAX+0.5, LAT_MIN-0.5, LAT_MAX+0.5)
bmap = plot_coor_on_map(bmap,  
                        train_df_small.loc[on_land, "pickup_longitude"].values,
                        train_df_small.loc[on_land, "pickup_latitude"].values,
                        marker="o", s=1, c="k", label=None)
ax[0].set_title("On Land")

bmap = create_nyc_map(ax[1], LON_MIN-0.5, LON_MAX+0.5, LAT_MIN-0.5, LAT_MAX+0.5)
bmap = plot_coor_on_map(bmap,  
                        train_df_small.loc[on_water, "pickup_longitude"].values,
                        train_df_small.loc[on_water, "pickup_latitude"].values,
                        marker="o", s=1, c="k", label=None)
ax[1].set_title("On Water")

How is `fare_amount` distributed among normal taxi trips and taxi trips which start and/or end up in water?

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
sns.kdeplot(train_df_small.loc[train_df_small["on_land"], "fare_amount"], ax=ax, label="On land")
sns.kdeplot(train_df_small.loc[~train_df_small["on_land"], "fare_amount"], ax=ax, label="On water")
ax.set_xlim([0, 100])
ax.set_xlabel("Fare (USD)")
ax.set_ylabel("Density")

The distribution of fare does not seem to be affected by whether pickup and/or dropoff locations are on land or on water. 

Let's compare distance distributions of normal trips and trips start and/or end up in water. The function below is based on https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula

In [ ]:
# return distance in kilometer
def distance(lon1, lat1, lon2, lat2):
    if isinstance(lon1, pd.Series):
        lon1 = lon1.values
    
    if isinstance(lat1, pd.Series):
        lat1 = lat1.values
        
    if isinstance(lon2, pd.Series):
        lon2 = lon2.values
    
    if isinstance(lat2, pd.Series):
        lat2 = lat2.values
        
    # use more precise floating numbers
    if isinstance(lon1, np.ndarray):
        lon1 = np.asarray(lon1, dtype=np.float64)
    
    if isinstance(lat1, np.ndarray):
        lat1 = np.asarray(lat1, dtype=np.float64)
        
    if isinstance(lon2, np.ndarray):
        lon2 = np.asarray(lon2, dtype=np.float64)
    
    if isinstance(lat2, np.ndarray):
        lat2 = np.asarray(lat2, dtype=np.float64)
        
    lon1_rad = np.radians(lon1)
    lat1_rad = np.radians(lat1)
    lon2_rad = np.radians(lon2)
    lat2_rad = np.radians(lat2)
    
    a = 0.5 - 0.5*np.cos(lat2_rad - lat1_rad) + np.cos(lat1_rad)*np.cos(lat2_rad)*(1 - np.cos(lon2_rad - lon1_rad))*0.5
    return 12742 * np.arcsin(np.sqrt(a))

def add_distance_col(df):
    df["distance"] = distance(df["pickup_longitude"], df["pickup_latitude"],
                              df["dropoff_longitude"], df["dropoff_latitude"])
    df["distance"] = df["distance"].astype(np.float32)
    return df

In [ ]:
train_df_small = add_distance_col(train_df_small)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
sns.kdeplot(train_df_small.loc[train_df_small["on_land"], "distance"], ax=ax, label="On land")
sns.kdeplot(train_df_small.loc[~train_df_small["on_land"], "distance"], ax=ax, label="On water")
ax.set_xlim([0, 40])
ax.set_xlabel("Distance (km)")
ax.set_ylabel("Density")

The distributions don't look that much different.

Anyway, there are only about 0.3% of the taxi rides having pickup and/or dropoff locations on water and determining them are very computationally expensive. So we will ignore them. 

In [ ]:
del train_df_small

## Clean up number of passenger

In [ ]:
print("min", train_df["passenger_count"].min())
print("max", train_df["passenger_count"].max())

In [ ]:
for c in [5, 6, 7, 10, 20, 50, 100]:
    print("Number of rows with passenger_count greater than %d is %d" %(c, (train_df["passenger_count"] > c).sum()))

We decide to remove rows having `passenger_count` greater than 6.

In [ ]:
print("shape before dropping", train_df.shape)
train_df = train_df[train_df["passenger_count"] <= 6]
print("shape after dropping", train_df.shape)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
df_tmp = train_df["passenger_count"].value_counts().sort_index()
df_tmp.plot(kind="bar", ax=ax)

ax.set_xlabel("passenger_count")
ax.set_ylabel("frequency")

xticklabels = df_tmp.index.to_list()
ax.set_xticklabels(xticklabels, rotation=0)

Let's see if `passenger_count` is related to `fare_amount`

In [ ]:
df_tmp = train_df.sample(frac=0.1, random_state=210)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

ax.scatter(df_tmp["passenger_count"], df_tmp["fare_amount"], s=1)
ax.set_ylim([0, 500])
ax.set_xlabel("passenger_count")
ax.set_ylabel("Fare (USD)")

del df_tmp

## Relationship between distance and fare

In [ ]:
train_df = add_distance_col(train_df)

In [ ]:
df_tmp = train_df.sample(frac=0.1, random_state=210)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

ax[0].scatter(df_tmp["distance"], df_tmp["fare_amount"], s=2)
ax[0].set_xlabel("Distance (km)")
ax[0].set_ylabel("Fare (USD)")
ax[0].set_title("Including all fare values")

df_tmp = df_tmp[df_tmp["fare_amount"] <= 1000]
ax[1].scatter(df_tmp["distance"], df_tmp["fare_amount"], s=2)
ax[1].set_xlabel("Distance (km)")
ax[1].set_ylabel("Fare (USD)")
ax[1].set_title("Including only fare at most $1000")

del df_tmp

In [ ]:
print("Number of points with fare_amount greater than $500 is %d" % (train_df["fare_amount"] > 500).sum())

Data points having `fare_amount` greater than $500 are rare and likely to be mistake. So we decide to drop them.

In [ ]:
print("Shape before dropping", train_df.shape)
train_df = train_df[train_df["fare_amount"] <= 500]
print("Shape after dropping", train_df.shape)

There seems to be many taxi trips with zero distance.

In [ ]:
print("There are %0.3f percent of rows having distance zero." % ((train_df["distance"] == 0).mean()*100))

Zero-distance trips happen when the taxi stays at the same place, i.e., pickup and dropoff locations are the same?

Let's compare fare distributions for the cases of zero and nonzero distances.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

sns.kdeplot(train_df.loc[train_df["distance"] == 0, "fare_amount"], ax=ax[0], label="zero distance")
sns.kdeplot(train_df.loc[train_df["distance"] > 0, "fare_amount"], ax=ax[0], label="nonzero distance")

ax[0].set_xlabel("Fare (USD)")
ax[0].set_ylabel("Density")
ax[0].set_title("Full range of fare_amount")

sns.kdeplot(train_df.loc[train_df["distance"] == 0, "fare_amount"], ax=ax[1], label="zero distance")
sns.kdeplot(train_df.loc[train_df["distance"] > 0, "fare_amount"], ax=ax[1], label="nonzero distance")

ax[1].set_xlim([0, 100])
ax[1].set_xlabel("Fare (USD)")
ax[1].set_ylabel("Density")
ax[1].set_title("Zoom-in at small fare_amount")

In [ ]:
print("Median fare of non-zero distance trips: $%0.2f" % (
    train_df.loc[train_df["distance"] > 0, "fare_amount"].median()))
print("Median fare of zero distance trips: $%0.2f" % (
    train_df.loc[train_df["distance"] == 0, "fare_amount"].median()))

In [ ]:
print("Min fare of zero distance trips: $%0.2f" % (
    train_df.loc[train_df["distance"] == 0, "fare_amount"].min()))
print("Max fare of zero distance trips: $%0.2f" % (
    train_df.loc[train_df["distance"] == 0, "fare_amount"].max()))

Zero-distance trips may correspond to those in which the customer canceled the ride and the driver charged for the waiting time. But the fact that there are very expensive Zero-distance, like $500 is really strange.

When doing modelling with machine learning algorithms, we may add a column to indicate whether a trip is zero-distance.

There are also zero-dollar trips which may be due to discount. 

In [ ]:
print("Number of rows with zero fare_amount is %d" % (train_df["fare_amount"] == 0).sum() )

Let's zoom in the `fare_amount` versus `distance` scatter plot to see more clearly the relationship. 

In [ ]:
df_tmp = train_df.sample(frac=0.1, random_state=210)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

ax.scatter(df_tmp["distance"], df_tmp["fare_amount"], s=1)
ax.set_xlabel("Distance (km)")
ax.set_ylabel("Fare (USD)")
ax.set_xlim(0, 30)
ax.set_ylim(0, 100)

del df_tmp

## Trips from or to airports

There are many horizontal stripes which may correspond to fixed-rate trips. These may be trips to or from airports. The city of New York is served by three major airports: **John F. Kennedy International Airport (JFK)**, **Newark Liberty International Airport (EWR)** and **LaGuardia Airport (LGA)**. We will mark each taxi ride in the training set with `No` if it does not involve being from and to an airport or with the airport code (`JFK`, `EWR` or `LGA`) to indicate that the trip is either from or to that airport.

A taxi trip is marked as being from or to an airport if either its pickup or dropoff loaction is within 3 km from the geo location of the airport. The geo locations of the three airports above can be found here:

https://tools.wmflabs.org/geohack/geohack.php?pagename=John_F._Kennedy_International_Airport&params=40_38_23_N_073_46_44_W_region:US-NY_type:airport

https://tools.wmflabs.org/geohack/geohack.php?pagename=LaGuardia_Airport&params=40_46_38.1_N_73_52_21.4_W_region:US-NY_type:airport

https://tools.wmflabs.org/geohack/geohack.php?pagename=Newark_Liberty_International_Airport&params=40_41_33_N_074_10_07_W_region:US-NJ_type:airport

In [ ]:
# 40.639722, -73.778889
JFK_LON = -73.778889
JFK_LAT = 40.639722

# 40.6925, -74.168611
EWR_LON = -74.168611
EWR_LAT = 40.6925

# 40.77725, -73.872611
LGA_LON = -73.872611
LGA_LAT = 40.77725

def is_to_airport(df, airport_lon, airport_lat, thres=3):
    dist = distance(airport_lon, airport_lat, df["dropoff_longitude"], df["dropoff_latitude"])
    return dist < thres

def is_from_airport(df, airport_lon, airport_lat, thres=3.):
    dist = distance(airport_lon, airport_lat, df["pickup_longitude"], df["pickup_latitude"])
    return dist < thres

def mark_airport_trip(df, 
                      jfk_lon=JFK_LON, jfk_lat=JFK_LAT, 
                      ewr_lon=EWR_LON, ewr_lat=EWR_LAT,
                      lga_lon=LGA_LON, lga_lat=LGA_LAT):
    
    df["from_to_airport"] = "No"
    
    from_to_jfk = is_to_airport(df, jfk_lon, jfk_lat) | is_from_airport(df, jfk_lon, jfk_lat)
    df.loc[from_to_jfk, "from_to_airport"] = "JFK"
    
    from_to_ewr = is_to_airport(df, ewr_lon, ewr_lat) | is_from_airport(df, ewr_lon, ewr_lat)
    df.loc[from_to_ewr, "from_to_airport"] = "EWR"
    
    from_to_lga = is_to_airport(df, lga_lon, lga_lat) | is_from_airport(df, lga_lon, lga_lat)
    df.loc[from_to_lga, "from_to_airport"] = "LGA"
    
    return df

In [ ]:
train_df = mark_airport_trip(train_df)

In [ ]:
train_df.head()

In [ ]:
train_df["from_to_airport"].value_counts()

In [ ]:
df_le_100 = train_df[train_df["fare_amount"] <= 100]

airports = df_le_100["from_to_airport"].value_counts().index

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))
plt.subplots_adjust(hspace=0.3)
axes = axes.flatten()

for airp, ax in zip(airports, axes):
    df_le_100.loc[df_le_100["from_to_airport"] == airp, "fare_amount"].plot(kind="hist", bins=50, ax=ax)
    ax.set_xlabel("Fare (USD)")
    ax.set_title(airp)

del df_le_100

Only trips to or from JFK show clearly the existence of fixed taxi rates. Nevetheless, the fare distributions look very different for the 4 cases.

In [ ]:
df_tmp = train_df.sample(frac=0.1, random_state=210)
print(df_tmp["from_to_airport"].value_counts())

airports = df_tmp["from_to_airport"].value_counts().index

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))
plt.subplots_adjust(hspace=0.3)
axes = axes.flatten()

for airp, ax in zip(airports, axes):
    ax.scatter(df_tmp.loc[df_tmp["from_to_airport"] == airp, "distance"],
               df_tmp.loc[df_tmp["from_to_airport"] == airp, "fare_amount"], 
               s=1)
    ax.set_xlabel("Distance (km)")
    ax.set_ylabel("Fare (USD)")
    ax.set_xlim(0, 30)
    ax.set_ylim(0, 100)
    ax.set_title(airp)
    
del df_tmp

There seems to be many fixed-rate taxi rides which do not involve airports. 

## Pickup and dropoff locations relative the city center.

In [ ]:
# New York City Coordinates from google
NYC_LON = -74.006
NYC_LAT = 40.7128

def add_pickup_to_center_dist_col(df, nyc_lon=NYC_LON, nyc_lat=NYC_LAT):
    df["pickup_to_center_dist"] = distance(nyc_lon, nyc_lat, 
                                           df["pickup_longitude"], df["pickup_latitude"])
    df["pickup_to_center_dist"] = df["pickup_to_center_dist"].astype(np.float32)
    return df


def add_dropoff_to_center_dist_col(df, nyc_lon=NYC_LON, nyc_lat=NYC_LAT):
    df["dropoff_to_center_dist"] = distance(nyc_lon, nyc_lat, 
                                            df["dropoff_longitude"], df["dropoff_latitude"])
    df["dropoff_to_center_dist"] = df["dropoff_to_center_dist"].astype(np.float32)
    return df

In [ ]:
train_df = add_pickup_to_center_dist_col(train_df)
train_df = add_dropoff_to_center_dist_col(train_df)

In [ ]:
df_tmp = train_df[train_df["pickup_to_center_dist"] < 50].sample(frac=0.2, random_state=210)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
plt.subplots_adjust(wspace=0.3)

df_tmp["pickup_to_center_dist"].plot(kind="hist", bins=100, ax=ax[0])
ax[0].set_xlabel("pickup-to-center distance (km)")
del df_tmp

df_tmp = train_df[train_df["dropoff_to_center_dist"] < 50].sample(frac=0.2, random_state=210)

df_tmp["dropoff_to_center_dist"].plot(kind="hist", bins=100, ax=ax[1])
ax[1].set_xlabel("dropoff-to-center distance (km)")
del df_tmp

The side peaks at about 15 and 20 km may be airports. Let's check that.

In [ ]:
print("Distance from NYC center to JFK airport is %0.2f km" % distance(NYC_LON, NYC_LAT, JFK_LON, JFK_LAT))

print("Distance from NYC center to EWR airport is %0.2f km" % distance(NYC_LON, NYC_LAT, EWR_LON, EWR_LAT))

print("Distance from NYC center to LGA airport is %0.2f km" % distance(NYC_LON, NYC_LAT, LGA_LON, LGA_LAT))

In [ ]:
df_tmp = train_df.sample(frac=0.2, random_state=210)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

ax.scatter(df_tmp["pickup_to_center_dist"], df_tmp["fare_amount"], s=1)
ax.set_xlabel("pickup-to-center distance (km)")
ax.set_ylabel("Fare (USD)")
del df_tmp

Let's also define `fare_per_km`. There are zero value of distance. So to avoid dividing by zero we replace zero distances with minimum value of the non-zero distances divided by 10.

In [ ]:
def fare_per_km(df):
    nonzero_dist_min = df.loc[df["distance"] > 0, "distance"].min()
    zero_impute_val = nonzero_dist_min / 10.
    print("Impute zero distance by %0.5f km" % zero_impute_val)
    
    dist = df["distance"].copy()
    dist[dist == 0] = zero_impute_val
    
    return df["fare_amount"] / dist

In [ ]:
train_df["fare_per_km"] = fare_per_km(train_df)

In [ ]:
df_tmp = train_df.sample(frac=0.2, random_state=210)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

df_tmp["fare_per_km"].plot(kind="hist", bins=100, ax=ax[0])
ax[0].set_xlabel("fare per km (USD/km)")
ax[0].set_title("Full range")
del df_tmp

df_tmp = train_df[train_df["fare_per_km"] < 20].sample(frac=0.2, random_state=210)
df_tmp["fare_per_km"].plot(kind="hist", bins=100, ax=ax[1])
ax[1].set_xlabel("fare per km (USD/km)")
ax[1].set_title("Zoom-in at small values")
del df_tmp

Let's see how pickup or dropoff location affects `fare_per_km`.

In [ ]:
df_tmp = train_df[train_df["fare_per_km"] < 20].sample(frac=0.2, random_state=210)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

ax.scatter(df_tmp["pickup_to_center_dist"], df_tmp["fare_per_km"], s=1)
ax.set_xlabel("pickup-to-center distance (km)")
ax.set_ylabel("fare per km (USD/km)")
del df_tmp

It looks like when pickup locations are near the center (`pickup_to_center_dist` < 25km), the fare per km does not depend on where the passengers are picked up. But further from the center the fare per km tends to decrease.

## Travel direction

Let's see how travel direction affects the taxi fare. Direction is measured by the angle made by the vector pointing from pickup to dropoff with the horizontal x axis. 

In [ ]:
def direction(lons_1, lats_1, lons_2, lats_2):
    bm = Basemap()
    
    x1, y1 = bm(lons_1, lats_1)
    x2, y2 = bm(lons_2, lats_2)
    
    dx = x2 - x1
    dy = y2 - y1
    
    hypotenuse = np.sqrt(dx*dx + dy*dy)
    
    pos_dx = dx >= 0
    neg_dx = dx < 0
    
    pos_dy = dy >= 0
    neg_dy = dy < 0
    
    neg_dx_and_pos_dy = neg_dx & pos_dy
    neg_dx_and_neg_dy = neg_dx & neg_dy
    
    direc = np.zeros(len(dx))
    
    direc[pos_dx] = np.arcsin(dy[pos_dx] / hypotenuse[pos_dx])
    
    direc[neg_dx_and_pos_dy] = np.pi - np.arcsin(dy[neg_dx_and_pos_dy] / hypotenuse[neg_dx_and_pos_dy])
    
    direc[neg_dx_and_neg_dy] = -np.pi - np.arcsin(dy[neg_dx_and_neg_dy] / hypotenuse[neg_dx_and_neg_dy])
    
    direc = 180 / np.pi * direc
    return direc

def add_direction_col(df):
    df["direction"] = direction(df["pickup_longitude"], df["pickup_latitude"],
                                df["dropoff_longitude"], df["dropoff_latitude"])
    df["direction"] = df["direction"].astype(np.float32)
    
    return df

In [ ]:
train_df = add_direction_col(train_df)

In [ ]:
print("Number of na %d" % train_df["direction"].isna().sum())
print("Percent of na %0.5f" % (train_df["direction"].isna().mean()*100))

NaN values are due to the distance (hypotenuse) being zero.

In [ ]:
print("Number of zero distances %d" %(train_df["distance"] == 0).sum())

Let's impute `NaN` values of `direction` with the median.

In [ ]:
def impute_nan_direction(df):
    df.loc[df["direction"].isna(), "direction"] = df["direction"].median()
    return df

In [ ]:
train_df = impute_nan_direction(train_df)

In [ ]:
train_df["direction"].isna().sum()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

train_df["direction"].plot(kind="hist", bins=100, ax=ax)
ax.set_xlabel("Direction angle (deg)")
ax.set_title("Histogram of direction angle")

Taxi trips along Northeast (about 60 deg) <--> Southwest (-140 deg) dominate. This is the direction of most streets in NYC. Let's look at the map zoom-in around Manhattan.

In [ ]:
# -74.05, -73.85, 40.7, 40.9
lon_min = -74.05
lon_max = -73.85
lat_min = 40.7
lat_max = 40.85

df_tmp = train_df.sample(frac=0.2, random_state=210)

sns.set(font_scale=1)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 8))

bmap = create_nyc_map(ax, lon_min, lon_max, lat_min, lat_max, resolution="f")
bmap = plot_coor_on_map(bmap,  
                        df_tmp["pickup_longitude"].values,
                        df_tmp["pickup_latitude"].values,
                        marker="o", s=0.001, c="k", alpha=0.3, label=None)
del df_tmp

Let's look at how `fare_amount` depends on `direction`.

In [ ]:
df_tmp = train_df.sample(frac=0.2, random_state=210)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

ax.scatter(df_tmp["direction"], df_tmp["fare_amount"], s=1)
ax.set_xlabel("Direction angle (deg)")
ax.set_ylabel("Fare (USD)")
del df_tmp

Let's bin the `direction` angle and calculate median `fare_amount` for each bin.

In [ ]:
bin_edges = np.linspace(-180, 180, 37)
bin_cent = (bin_edges[:-1] + bin_edges[1:])/2

direc_cut = pd.cut(train_df["direction"], bin_edges)
median_fare_by_direc = train_df.groupby(direc_cut)["fare_amount"].median()
median_fare_by_direc.index = bin_cent

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))
median_fare_by_direc.plot(kind="line", ax=ax, linestyle="-", marker=".")
ax.set_xlabel("Direction angle (deg)")
ax.set_ylabel("Median fare (USD)")

## Cleaning and exploring temporal data

Let's create columns `year`, `month`, `weekday` and `hour` from column `pickup_datetime`

In [ ]:
t = train_df["pickup_datetime"]

In [ ]:
t.apply(lambda t: t.weekday)[:10]

In [ ]:
def ad_time_cols(df):
    df["year"] = df["pickup_datetime"].apply(lambda t: t.year).astype(np.int32)
    df["month"] = df["pickup_datetime"].apply(lambda t: t.month).astype(np.uint8)
    df["weekday"] = df["pickup_datetime"].apply(lambda t: t.weekday).astype(np.uint8)
    df["hour"] = df["pickup_datetime"].apply(lambda t: t.hour).astype(np.uint8)
    return df

In [ ]:
train_df = ad_time_cols(train_df)

In [ ]:
train_df.head()

In [ ]:
print(train_df["year"].isna().sum())
print(train_df["year"].min())
print(train_df["year"].max())

In [ ]:
print(train_df["month"].isna().sum())
print(train_df["month"].min())
print(train_df["month"].max())

In [ ]:
print(train_df["weekday"].isna().sum())
print(train_df["weekday"].min())
print(train_df["weekday"].max())

In [ ]:
print(train_df["hour"].isna().sum())
print(train_df["hour"].min())
print(train_df["hour"].max())

Data may not have been collected for the whole year in 2015. Fare slighly increases over the year.

## Taxi usage and fare by year, month, weekday and hour.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))
plt.subplots_adjust(hspace=0.3, wspace=0.3)
axes = axes.flatten()

for ax, t in zip(axes, ["year", "month", "weekday", "hour"]):
    agg = train_df.groupby(t)["fare_amount"].agg("count")
    agg.plot(kind="bar", ax=ax)
    ax.set_ylabel("Total number of trips")
    ax.set_xticklabels(agg.index.to_list(), rotation=0)

Data was not collected for the whole year of 2015. So let's remove 2015 in calculating count statistic. 

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))
plt.subplots_adjust(hspace=0.3, wspace=0.3)
axes = axes.flatten()

df_tmp = train_df[train_df["year"] != 2015]

for ax, t in zip(axes, ["year", "month", "weekday", "hour"]):
    agg = df_tmp.groupby(t)["fare_amount"].agg("count")
    agg.plot(kind="bar", ax=ax)
    ax.set_ylabel("Total number of trips")
    ax.set_xticklabels(agg.index.to_list(), rotation=0)

del df_tmp

During a year the number of taxi trips drops in August which maybe due to summer break. During a week, people use less taxi in on Sunday which makes sense. During a day, the peak time is from 6pm to 10pm. 

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))
plt.subplots_adjust(hspace=0.3, wspace=0.3)
axes = axes.flatten()

for ax, t in zip(axes, ["year", "month", "weekday", "hour"]):
    agg = train_df.groupby(t)["fare_amount"].agg("median")
    agg.plot(kind="bar", ax=ax)
    ax.set_ylabel("Median fare (USD)")
    ax.set_xticklabels(agg.index.to_list(), rotation=0)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))
plt.subplots_adjust(hspace=0.3, wspace=0.3)
axes = axes.flatten()

for ax, t in zip(axes, ["year", "month", "weekday", "hour"]):
    agg = train_df.groupby(t)["fare_per_km"].agg("median")
    agg.plot(kind="bar", ax=ax)
    ax.set_ylabel("Median fare per km (USD)")
    ax.set_xticklabels(agg.index.to_list(), rotation=0)

Fare increases over year. Fare is also low in summer and on weekend.

## Save to csv

We remove columns generated during EDA before saving to csv file.

In [ ]:
train_df.columns

In [ ]:
rm_cols = train_df.columns.to_list()[7:]
rm_cols 

In [ ]:
train_df.drop(rm_cols, axis=1, inplace=True)
train_df.shape

In [ ]:
save_csv = False

if save_csv:
    train_df.to_csv(os.path.join(PROC_DIR, "train_df.csv"), index=False)

In [ ]:
save_csv = False

if save_csv:
    test_df.to_csv(os.path.join(PROC_DIR, "test_df.csv"), index=False)

In [ ]:
del train_df
del test_df